In [10]:
!pip install -q asf_search
!pip install -q geopandas
!pip install -q geobr

     |████████████████████████████████| 928 kB 12.9 MB/s 


In [29]:
# Bibliotecas
import asf_search as asf
import tifffile as tif
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import box
from datetime import date
import geobr
import numpy as np

In [12]:
# Baixando vetor do Distrito Federal
DF = geobr.read_state(code_state="DF", year=2020)

In [15]:
# Extraindo coordenadas de canto
bounds = DF.total_bounds

# Transformando coordenadas em GeoDataFrame
bounds_gdf = gpd.GeoSeries([box(*bounds)])

# Convertendo coordenadas para WKT
wkt = bounds_gdf.to_wkt().values.tolist()[0]

In [31]:
# Realizando pesquisa por cenas S1-GRD
results = asf.search(
    platform= asf.PLATFORM.SENTINEL1A,
    processingLevel=[asf.PRODUCT_TYPE.GRD_HD],
    start = date(2022, 1, 1),
    end = date(2022, 1, 31),
    intersectsWith = wkt
    )

print(f'Número de cenas: {len(results)}')

Número de cenas: 2


In [17]:
# Criando sessão
session = asf.ASFSession().auth_with_creds('Usuário', 'Senha')

In [23]:
# Baixando cenas
results.download(
     path = '/content',
     session = session, 
     processes = 2 
  )

In [ ]:
# Descompactando arquivo
!unzip /content/S1A_IW_GRDH_1SDV_20220110T084531_20220110T084556_041396_04EC0D_C3A1.zip

In [27]:
# Função de expansão
def expansao(img, percent_ini, percent_fim):
    s = np.zeros_like(img)
    x,y = 0,255  
    w = np.percentile(img, percent_ini)
    z = np.percentile(img, percent_fim)        
    p = x + (img - w) * (y - x) / (z - w)    
    p[p<x] = x
    p[p>y] = y
    s = p
    return s

In [25]:
# Lendo polarização VV
img = tif.imread('caminho da imagem descompactada')

In [ ]:
# Visualizando resultado
plt.figure(figsize=(12,8))
plt.imshow(expansao(img, 2, 98), cmap='gray')